In [1]:
!git clone https://github.com/anonymousindividual007/Multi-environment-Topic-Models

Cloning into 'Multi-environment-Topic-Models'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 82 (delta 40), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (82/82), 42.64 MiB | 9.59 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Updating files: 100% (17/17), done.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import statsmodels.api as sm
import zipfile
import os

The topic_proportions folder contains the topic proportions from other baseline models. Change the zipfile path to access them. In a couple cells below you should also change the topic proportion you index into.

In [3]:
syn_experiment='senior'

In [4]:
temp_dir = '/content/temp_dir'

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

if syn_experiment=='energy':

    zip_file_path_1 = '/content/Multi-environment-Topic-Models/VTM_train_topic_proportions_lda.csv.zip'
    csv_file_name_1 = 'VTM_train_topic_proportions_lda.csv'

    zip_file_path_2 = '/content/Multi-environment-Topic-Models/eatm2_topic_proportions.csv.zip'
    csv_file_name_2 = 'eatm2_topic_proportions.csv'

    with zipfile.ZipFile(zip_file_path_1, 'r') as zip_ref:
        zip_ref.extract(csv_file_name_1, temp_dir)

    with zipfile.ZipFile(zip_file_path_2, 'r') as zip_ref:
        zip_ref.extract(csv_file_name_2, temp_dir)

    csv_file_path_1 = os.path.join(temp_dir, csv_file_name_1)
    csv_file_path_2 = os.path.join(temp_dir, csv_file_name_2)

    mtm = pd.read_csv(csv_file_path_2)
    vtm = pd.read_csv(csv_file_path_1)


else:

    zip_file_path_1 = '/content/Multi-environment-Topic-Models/VTM_tp_immi.csv.zip'
    csv_file_name_1 = 'VTM_tp_immi.csv'

    zip_file_path_2 = '/content/Multi-environment-Topic-Models/2mtm_medicare_ss_topics.csv.zip'
    csv_file_name_2 = '2mtm_medicare_ss_topics.csv'

    with zipfile.ZipFile(zip_file_path_1, 'r') as zip_ref:
        zip_ref.extract(csv_file_name_1, temp_dir)

    with zipfile.ZipFile(zip_file_path_2, 'r') as zip_ref:
        zip_ref.extract(csv_file_name_2, temp_dir)

    csv_file_path_1 = os.path.join(temp_dir, csv_file_name_1)
    csv_file_path_2 = os.path.join(temp_dir, csv_file_name_2)

    mtm = pd.read_csv(csv_file_path_2)
    vtm = pd.read_csv(csv_file_path_1)



data = pd.read_csv('/content/Multi-environment-Topic-Models/ci_mtm_senior_energy.csv')

In [7]:
mtm = mtm.drop(columns=['Unnamed: 0'])

Uncomment for MTM proportions from the energy experiment

In [8]:
#energy
# def process_row_mtm(row):
#     if row['Topic_28'] == max(row):
#         return 1
#     else:
#         return 0

# mtm['binary 28'] = mtm.apply(process_row_mtm, axis=1)

Senior experiment: MTM

In [20]:
def process_row_mtm(row):
    if row['Topic_15'] == max(row):
        return 1
    else:
        return 0

mtm['binary topic'] = mtm.apply(process_row_mtm, axis=1)

Energy topic for baseline models. The different values we index row with represent the topic corresponding to energy or senior social policies for each experiment. Uncomment the one that corresponds to which model you would like to evaluate.

In [21]:
#vtm
# def process_row_vtm_15(row):
#     if row['15'] == max(row):
#         return 1
#     else:
#         return 0
# vtm_topics['binary 15'] = vtm_topics.apply(process_row_vtm_15, axis=1)


#lda
# def process_row_vtm_15(row):
#     if row.idxmax() == 'Topic_11':
#         return 1
#     else:
#         return 0

#prodlda
# def process_row_vtm_15(row):
#     if row.idxmax() == 'Topic_1':
#         return 1
#     else:
#         return 0

#bertopic
# def process_row_vtm_15(row):
#     if row.idxmax() == 'Topic_19':
#         return 1
#     else:
#         return 0



Senior topic for baseline models

In [22]:
#senior
#prodlda
# def process_row_vtm_15(row):
#     if row.idxmax() == 'Topic_21':
#         return 1
#     else:
#         return 0

#gensim
# def process_row_vtm_15(row):
#     if row.idxmax() == 'Topic_8':
#         return 1
#     else:
#         return 0

#vtm
def process_row_vtm_15(row):
    if row.idxmax() == 'Topic_0':
        return 1
    else:
        return 0

# #bertopic
# def process_row_vtm_15(row):
#     if row.idxmax() == 'Topic_2':
#         return 1
#     else:
#         return 0


vtm['binary 15'] = vtm.apply(process_row_vtm_15, axis=1)



Combined topics for the VTM in the energy experiment

In [23]:
# def process_row_vtm(row):
#     max_value = max(row)
#     # Check if the maximum value corresponds to either '15' or '21'
#     if row['15'] == max_value or row['21'] == max_value:
#         return 1
#     else:
#         return 0

# vtm_topics['binary topic'] = vtm_topics.apply(process_row_vtm, axis=1)

In [24]:
# Create an indicator variable for the 'source' column
data['source_indicator'] = data['source'].apply(lambda x: 1 if x == 'Republican' else 0)

In [25]:
merged_df_vtm = data.merge(vtm, left_index=True, right_index=True)
merged_df_mtm = data.merge(mtm, left_index=True, right_index=True)


Change is_senior to is_energy for the energy experiment

In [26]:
#sampling senior
senior_true_sample_2 = merged_df_mtm[merged_df_mtm['is_senior'] == True].sample(n=700, random_state=1)
senior_false_sample_2 = merged_df_mtm[merged_df_mtm['is_senior'] == False].sample(n=700, random_state=1)
final_mtm = pd.concat([senior_true_sample_2, senior_false_sample_2])

In [27]:
senior_true_sample_1 = merged_df_vtm[merged_df_vtm['is_senior'] == True].sample(n=700, random_state=1)
senior_false_sample_1 = merged_df_vtm[merged_df_vtm['is_senior'] == False].sample(n=700, random_state=1)
final_vtm = pd.concat([senior_true_sample_1, senior_false_sample_1])

In [28]:
baseline_prob = 0.5


In [29]:
def adjust_probability(row, baseline_prob=baseline_prob, seed=None):
    # if row['is_energy']:
    if row['is_senior']:

        noise = np.random.uniform(0, 0.1)
        adjusted_prob = min(baseline_prob + 0.2 + noise, 1)
        return np.random.binomial(1, adjusted_prob)
    else:
        return np.random.binomial(1, baseline_prob)

# Initialize oucomes with a fixed seed for reproducibility
np.random.seed(10)
final_vtm['outcome'] = np.random.binomial(1, baseline_prob, size=len(final_vtm))
final_mtm['outcome']=np.random.binomial(1, baseline_prob, size=len(final_mtm))

final_vtm['outcome'] = final_vtm.apply(adjust_probability, axis=1)
final_mtm['outcome'] = final_mtm.apply(adjust_probability, axis=1)

In [30]:
X_2 = final_mtm[['binary topic', 'source_indicator']]
y_2 = final_mtm['outcome']
X_2 = sm.add_constant(X_2)
model_eatm2 = sm.OLS(y_2, X_2).fit(cov_type='HC3')
print(model_eatm2.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     34.55
Date:                Tue, 01 Oct 2024   Prob (F-statistic):           2.25e-15
Time:                        14:15:02   Log-Likelihood:                -948.92
No. Observations:                1400   AIC:                             1904.
Df Residuals:                    1397   BIC:                             1920.
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.5718      0.021  

In [31]:
vtm_X_2 = final_vtm[['binary 15', 'source_indicator']]
vtm_y_2 = final_vtm['outcome']
vX_2 = sm.add_constant(vtm_X_2)
model_vtm_2 = sm.OLS(vtm_y_2, vX_2).fit(cov_type='HC3')
print(model_vtm_2.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     4.971
Date:                Tue, 01 Oct 2024   Prob (F-statistic):             0.0259
Time:                        14:15:02   Log-Likelihood:                -964.60
No. Observations:                1400   AIC:                             1933.
Df Residuals:                    1398   BIC:                             1944.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.6553      0.017  

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
